In [1]:
import os
import re
import json
import pickle

import fasttext

import shutil

import time
from tqdm import tqdm
import math

import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau

from transformers import AutoTokenizer, AutoModel

In [2]:
words = np.load('/home/sju/HyoJun/Creative_semester_system/words.npy')
words = words.tolist()
y_data1 = np.load('/home/sju/HyoJun/Creative_semester_system/y_data1.npy')

In [3]:
class TrainDataset(Dataset):
    def __init__(self, words, landmarks):
        self.words = words
        self.landmarks = torch.tensor(landmarks, dtype=torch.float32)  # (N, T, L, D)

    def __len__(self):
        return len(self.words)

    def __getitem__(self, idx):
        word = self.words[idx]                # str or list of str
        landmark_seq = self.landmarks[idx]    # (T, L, D)
        return word, landmark_seq

class TestDataset(Dataset):
    def __init__(self, words, landmarks):
        self.words = words
        self.landmarks = torch.tensor(landmarks, dtype=torch.float32)
        self.first_frame = self.landmarks[:, 0, :, :]  # (N, L, D)

    def __len__(self):
        return len(self.words)

    def __getitem__(self, idx):
        word = self.words[idx]
        first_frame = self.first_frame[idx]   # (L, D)
        return word, first_frame

In [4]:
class WordEmbeddingDecoderModel(nn.Module):
    def __init__(self, ft_model_path, hidden_size, num_frames, num_landmarks, dim):
        super(WordEmbeddingDecoderModel, self).__init__()
        self.num_frames = num_frames
        self.num_landmarks = num_landmarks
        self.dim = dim
        self.hidden_size = hidden_size

        self.ft = fasttext.load_model(ft_model_path)
        self.embedding_dim = self.ft.get_dimension()
        
        self.encoder_proj = nn.Linear(self.embedding_dim, hidden_size)

        self.decoder_input_proj = nn.Linear(num_landmarks * dim, hidden_size)

        self.decoder_layer = nn.TransformerDecoderLayer(
            d_model=hidden_size,
            nhead=8,
            dim_feedforward=hidden_size//2,
            dropout=0.3,
            activation='relu',
            batch_first=True
        )
        self.transformer_decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=8)

        self.fc = nn.Linear(hidden_size, num_landmarks * dim)

    def encode_text(self, batch_words, device='cuda'):
        
        batch_embeddings = []
        for words in batch_words:
            word_embeds = [self.ft.get_word_vector(w) for w in words]
            batch_embeddings.append(word_embeds)
        
        embeddings = torch.tensor(batch_embeddings).to(device)
        embeddings = self.encoder_proj(embeddings)
        
        return embeddings
        

    def forward(self, words, decoder_input: torch.Tensor, device: torch.device):
        
        memory = self.encode_text(words, device)

        # Prepare decoder input
        decoder_input = decoder_input.to(device) 
        
        batch_size = decoder_input.size(0)
        num_frames = decoder_input.size(1)
        
        decoder_input_flat = decoder_input.view(batch_size, num_frames, -1)
        decoder_input_proj = self.decoder_input_proj(decoder_input_flat)  # (B, T, hidden_size)

        # Create autoregressive mask
        tgt_mask = torch.triu(torch.ones(num_frames, num_frames), diagonal=1).bool().to(device)

        # Decode
        decoder_output = self.transformer_decoder(
            tgt=decoder_input_proj,
            memory=memory,
            tgt_mask=tgt_mask
        )  # (B, T, hidden_size)

        output = self.fc(decoder_output)  # (B, T, L*D)
        output = output.view(batch_size, num_frames, self.num_landmarks, self.dim)

        return output

In [5]:
#MODEL = BERT
HIDDEN_SIZE = 768
NUM_FRAMES = y_data1.shape[1]
NUM_LANDMARKS = y_data1.shape[2]
DIM = 2

EPOCHS = 200
BATCH_SIZE = 1
LR = 1e-6

In [6]:
train_dataset = TrainDataset(words, y_data1)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [7]:
model = WordEmbeddingDecoderModel(
    ft_model_path='/home/sju/HyoJun/Creative_semester_system/cc.ko.300.bin',
    hidden_size=HIDDEN_SIZE,
    num_frames=NUM_FRAMES,
    num_landmarks=NUM_LANDMARKS,
    dim=DIM
).cuda()

In [8]:
class BoneLengthLoss(nn.Module):
    def __init__(self, bone_pairs):
        super(BoneLengthLoss, self).__init__()
        self.bone_pairs = bone_pairs
    
    def forward(self, pred, target):
        assert pred.shape == target.shape
        
        B, T, N, D = pred.shape
        pred_bones = []
        target_bones = []
        for i, j in self.bone_pairs:
            pred_bone = torch.norm(pred[:, :, i, :] - pred[:, :, j, :], dim=-1)
            target_bone = torch.norm(target[:, :, i, :] - target[:, :, j, :], dim=-1)
            pred_bones.append(pred_bone)
            target_bones.append(target_bone)
        
        pred_bones = torch.stack(pred_bones, dim=-1)
        target_bones = torch.stack(target_bones, dim=-1)
        
        loss = torch.mean((pred_bones - target_bones) ** 2)
        return loss

In [9]:
hand_bone_pairs = [
    (0, 1), (1, 2), (2, 3), (3, 4),      # Thumb
    (0, 5), (5, 6), (6, 7), (7, 8),      # Index
    (0, 9), (9,10), (10,11), (11,12),    # Middle
    (0,13), (13,14), (14,15), (15,16),   # Ring
    (0,17), (17,18), (18,19), (19,20)    # Pinky
]

pose_bone_pairs = [
    (0, 1),   # nose to neck
    (1, 2), (2, 3), (3, 4), (4, 5),   # right arm
    (1, 6), (6, 7), (7, 8), (8, 9),   # left arm
    (1, 10), (10, 11), (11, 12),     # right leg
    (1, 13), (13, 14), (14, 15),     # left leg
    (0, 16), (0, 17),  # eyes
    (16, 18), (17, 19),  # ears
    (11, 22), (14, 19),  # ankles to toes
]

In [10]:
#criterion = nn.L1Loss()
criterion = nn.MSELoss()
#criterion_hand = BoneLengthLoss(hand_bone_pairs)
#criterion_pose = BoneLengthLoss(pose_bone_pairs)

optimizer = AdamW(model.parameters(), lr=LR)

cos_epoch = int(EPOCHS*0.75)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, (EPOCHS-cos_epoch)*len(train_dataloader))

steps_per_epoch = len(train_dataloader)
#val_steps = len(valid_dataloader)

In [11]:
%%time

for epoch in tqdm(range(EPOCHS)):
    
    model.train()
    train_loss = 0
    t = time.time()
    
    for step, batch in enumerate(train_dataloader):
        word, landmark = batch
        word = word  # word는 list여서 GPU로 이동하지 않음
        landmark = landmark.cuda()  # landmark는 Tensor이므로 GPU로 이동
        
        decoder_input = landmark

        optimizer.zero_grad()
        
        output = model(word, decoder_input, 'cuda')
        
        loss = criterion(output, landmark) 
        #criterion(output[..., 0:70, :], landmark[..., 0:70, :]) + criterion_pose(output[..., 70:95, :], landmark[..., 70:95, :]) + criterion_hand(output[..., 95:116, :], landmark[..., 95:116, :]) + criterion_hand(output[..., 116:137, :], landmark[..., 116:137, :])
        loss.backward()

        optimizer.step()
            
        train_loss += loss.item()
        print ("Step [{}/{}] Loss: {:.3f} Time: {:.1f}".format(step+1, steps_per_epoch, train_loss/(step+1), time.time()-t),end='\r',flush=True)
        
        if epoch > cos_epoch:
            scheduler.step()
            
    print('')
    train_loss /= (step+1)

  0%|                                                   | 0/200 [00:00<?, ?it/s]

/tmp/ipykernel_40399/2973326957.py:35: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  embeddings = torch.tensor(batch_embeddings).to(device)


  0%|▏                                        | 1/200 [00:43<2:23:04, 43.14s/it]

  1%|▍                                        | 2/200 [01:25<2:20:50, 42.68s/it]

  2%|▌                                        | 3/200 [02:08<2:20:07, 42.68s/it]

...

 98%|██████████████████████████████████████▍| 197/200 [2:20:52<02:07, 42.65s/it]

 99%|██████████████████████████████████████▌| 198/200 [2:21:34<01:25, 42.70s/it]

100%|██████████████████████████████████████▊| 199/200 [2:22:18<00:42, 42.89s/it]

100%|███████████████████████████████████████| 200/200 [2:23:01<00:00, 42.91s/it]


CPU times: user 1d 2h 13min 5s, sys: 1min 49s, total: 1d 2h 14min 54s
Wall time: 2h 23min 1s


In [12]:
torch.save(model.state_dict(), f"/home/sju/HyoJun/Creative_semester_system/Model/text2sign1.pth")

In [13]:
words

[['고민'],
 ['뻔뻔'],
 ['수어'],
 ['남아'],
 ['눈'],
 ['독신'],
 ['음료수'],
 ['발가락'],
 ['슬프다'],
 ['자극'],
 ['안타깝다'],
 ['어색하다'],
 ['여아'],
 ['외국인'],
 ['영아'],
 ['신사'],
 ['뉴질랜드'],
 ['나사렛대학교'],
 ['알아서'],
 ['장애인'],
 ['열아홉번째'],
 ['침착'],
 ['성실'],
 ['학교연혁'],
 ['싫어하다'],
 ['급하다'],
 ['필기시험'],
 ['병문안'],
 ['검사'],
 ['결승전'],
 ['낚시터'],
 ['낚시대'],
 ['당뇨병'],
 ['독서'],
 ['매표소'],
 ['면역'],
 ['감기'],
 ['배드민턴'],
 ['변비'],
 ['병명'],
 ['보건소'],
 ['불면증'],
 ['불행'],
 ['붕대'],
 ['사위'],
 ['설사'],
 ['성병'],
 ['방충'],
 ['소화제'],
 ['손녀'],
 ['손자'],
 ['수면제'],
 ['수집가'],
 ['여행지'],
 ['예식장'],
 ['올림픽경기'],
 ['회복'],
 ['첫번째'],
 ['운동경기'],
 ['입원'],
 ['재혼'],
 ['진단서'],
 ['축구장'],
 ['치료'],
 ['치료법'],
 ['친아들'],
 ['퇴원'],
 ['한약'],
 ['한약방'],
 ['빈혈'],
 ['화상'],
 ['가래떡'],
 ['고깃국'],
 ['고추'],
 ['고추가루'],
 ['사골'],
 ['배추국'],
 ['꽈베기'],
 ['벌꿀'],
 ['꿀물'],
 ['냄비'],
 ['찬물'],
 ['다과'],
 ['지방경찰청장'],
 ['된장찌게'],
 ['돼지고기'],
 ['두부'],
 ['딸기'],
 ['떡국'],
 ['라면'],
 ['막걸리'],
 ['무'],
 ['밥그릇'],
 ['밥솥'],
 ['보신탕'],
 ['부엌'],
 ['소불고기'],
 ['비빔밥'],
 ['사과'],
 ['사이다'],
 ['달다'],
 ['소주'],
 ['술잔'],
 [

In [ ]:
n = 0

word = [words[n]]
landmark = y_data1[n, ...]

: 

: 

In [ ]:
print(word)

: 

: 

In [ ]:
landmark.shape

: 

: 

In [24]:
%%time

test_dataset = TestDataset(word, landmark[None, ...])
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

model.eval()

preds_list = []
for batch in test_dataloader:
    word, first_frame = batch
    word = word
    first_frame = first_frame.cuda()
    
    current_dec_input = first_frame.unsqueeze(1)   #.float()
    
    for step in range(NUM_FRAMES):  

        with torch.no_grad():
            output = model(word, current_dec_input, 'cuda')
            last_frame = output[:, -1, :, :]  # (B, L, D)
            last_frame_unsq = last_frame.unsqueeze(1)  # (B, 1, L, D)
            current_dec_input = torch.cat([current_dec_input, last_frame_unsq], dim=1)

    final_pred = current_dec_input[:, 1:, :, :]  # (batch, NUM_FRAMES, L, D)

    preds_list.append(final_pred.cpu())

preds = torch.cat(preds_list, dim=0).numpy()
print(preds.shape)

(1, 204, 137, 2)
CPU times: user 22 s, sys: 5.29 ms, total: 22 s
Wall time: 973 ms


In [25]:
with open('/home/sju/HyoJun/Creative_semester_system/scalers.pkl', 'rb') as f:
    scalers = pickle.load(f)

In [26]:
y_data_restored = []

for arr_normalized, scaler in zip(preds, scalers):
    arr_restored = scaler.inverse_transform(arr_normalized.reshape(-1, arr_normalized.shape[-1])).reshape(arr_normalized.shape)
    y_data_restored.append(arr_restored)

y_data_restored = np.array(y_data_restored)
print("Restored y_data shape:", y_data_restored.shape)

Restored y_data shape: (1, 204, 137, 2)


In [27]:
preds_inv = y_data_restored
print(preds_inv.shape)
np.save(f"/home/sju/HyoJun/Creative_semester_system/preds/sign_preds{n+1}.npy", preds_inv)

(1, 204, 137, 2)
